In [2]:
import pandas as pd
import numpy as np

In [3]:
# read and prind the data
df = pd.read_csv("Bird_strikes.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Bird_strikes.csv'

In [75]:
features = data.columns
print(features)

Index(['RecordID', 'AircraftType', 'AirportName', 'AltitudeBin', 'MakeModel',
       'NumberStruck', 'NumberStruckActual', 'Effect', 'FlightDate', 'Damage',
       'Engines', 'Operator', 'OriginState', 'FlightPhase',
       'ConditionsPrecipitation', 'RemainsCollected?',
       'RemainsSentToSmithsonian', 'Remarks', 'WildlifeSize', 'ConditionsSky',
       'WildlifeSpecies', 'PilotWarned', 'Cost', 'Altitude', 'PeopleInjured',
       'IsAircraftLarge?'],
      dtype='object')


In [76]:
species = data['WildlifeSpecies'].unique()

# Display the bird species mentioned on the data
print(species)

['Unknown bird - medium' 'Rock pigeon' 'European starling'
 'Unknown bird - small' 'Canada goose' 'Snow goose' 'Black-headed munia'
 'Ring-billed gull' 'Sandhill crane' 'Western meadowlark' 'Mallard'
 'Turkey vulture' 'Tundra swan' 'Unknown bird or bat' 'Killdeer'
 'Western sandpiper' 'Wild turkey' 'Lapland longspur' 'Herring gull'
 'Cape May warbler' 'American crow' 'Common myna' 'Unknown bird - large'
 'Red-winged blackbird' 'American kestrel' 'Mourning dove'
 'Eastern meadowlark' 'Upland sandpiper' 'Purple martin'
 'Brown-headed cowbird' 'American robin' 'Horned lark' 'Mute swan'
 'American pipit' 'Snow bunting' 'Gadwall' 'White-tailed deer'
 'Pacific golden-plover' 'Western gull' 'Barn swallow' 'Cattle egret'
 'Redhead' 'Common raven' 'Laughing gull' 'Bald eagle' 'Bank swallow'
 'Finches' 'Great-tailed grackle' 'Tree swallow' 'Great black-backed gull'
 'Black-bellied plover' 'Least tern' 'Common grackle' 'Great cormorant'
 'Cedar waxwing' 'Greater white-fronted goose' 'American woo

In [77]:
# Define the mapping functions and weights
size_mapping = {
    "Small": ["American kestrel", "Mourning dove", "Rock pigeon", "European starling", 
              "Common myna", "House sparrow", "Pigeons", "Zebra dove", "House finch",
              "Tree swallow", "Lark bunting", "Cedar waxwing", "Finches", "Purple martin",
              "Horned lark", "Red-vented bulbul", "Laughing gull", "Common raven", 
              "Spotted dove", "Cave swallow", "Common nighthawk", "Common waxbill", 
              "Red-eyed vireo", "Indigo bunting", "Purple finch", "Chipping sparrow", 
              "White-crowned sparrow", "Ruby-throated hummingbird", "Swallow", "Sparrows", "Unknown bird - small"],
    "Medium": ["Red-tailed hawk", "Herons, egrets, bitterns", "Mallard", "Turkey vulture", 
              "Snow goose", "Canada goose", "Sandhill crane", "Turkey vulture", 
              "Bald eagle", "Barn swallow", "Common raven", "American crow", "Wood stork", 
              "Great blue heron", "Northern harrier", "Osprey", "Red-shouldered hawk", 
              "Western grebe", "American woodcock", "Red-tailed kite", "American white pelican", 
              "Wild turkey", "Great egret", "Ducks", "Gulls", "Bald eagle","Unknown bird - medium"],
    "Large": ["Domestic dog", "Northern saw-whet owl", "Gray francolin", "Common loon", 
           "Great horned owl", "Mottled duck", "American black duck", "Coyote", "Unknown bird - large"],
    "Unknown": ["Unknown bird or bat","Unknown"]
}

def map_size(species):
    for size, species_list in size_mapping.items():
        if species in species_list:
            return size
    return "Unknown"  # Default if species not in mapping

size_weights = {"Small": 0.2, "Medium": 0.5, "Large": 1.0, "Unknown": 0.3}

weather_conditions = ["Clear", "Rain", "Fog", "Storm"]
weather_weights = {"Clear": 1, "Rain": 2, "Fog": 3, "Storm": 5}

alert_thresholds = [7, 15, 22]  # Example thresholds

# Add random weather conditions for demonstration
data["Weather_Condition"] = np.random.choice(weather_conditions, size=len(data))
data["Weather_Weight"] = data["Weather_Condition"].map(weather_weights)

# Filter the dataset to include only bird-related entries
bird_keywords = ["bird", "kestrel", "dove", "hawk", "egrets", "herons"]
data_birds = data[data["WildlifeSpecies"].str.contains('|'.join(bird_keywords), case=False, na=False)]

# Recompute relevant columns for the filtered data
data_birds["Bird_Size"] = data_birds["WildlifeSpecies"].apply(map_size)
data_birds["Bird_Count"] = data_birds["NumberStruckActual"]
data_birds["Size_Weight"] = data_birds["Bird_Size"].map(size_weights)
data_birds["Alert_Score"] = (
    data_birds["Weather_Weight"] +
    data_birds["Bird_Count"] * data_birds["Size_Weight"]
)
data_birds["Alert_Category"] = pd.cut(
    data_birds["Alert_Score"],
    bins=[-np.inf, alert_thresholds[0], alert_thresholds[1], alert_thresholds[2], np.inf],
    labels=["Low", "Medium", "High", "Critical"]
)

# Display the filtered dataset preview with the Bird Size categorization
data_birds_preview = data_birds[
    ["WildlifeSpecies", "Bird_Size", "Bird_Count", "Weather_Condition", 
     "Weather_Weight", "Size_Weight", "Alert_Score", "Alert_Category"]
]

print(data_birds_preview)

# Optionally, save the new dataset to a CSV file
data_birds_preview.to_csv('Risk_Analysis.csv', index=False)


             WildlifeSpecies Bird_Size  Bird_Count Weather_Condition  \
0      Unknown bird - medium    Medium         859             Clear   
5       Unknown bird - small     Small         537               Fog   
9       Unknown bird - small     Small           4             Clear   
12      Unknown bird - small     Small           2               Fog   
13      Unknown bird - small     Small           2             Storm   
...                      ...       ...         ...               ...   
25422   Unknown bird - small     Small           1             Storm   
25423   Unknown bird - small     Small           1             Storm   
25425   Unknown bird - large     Large           1             Storm   
25427  Unknown bird - medium    Medium           1             Storm   
25428        Red-tailed hawk    Medium           1             Clear   

       Weather_Weight  Size_Weight  Alert_Score Alert_Category  
0                   1          0.5        430.5       Critical  
5    

C:\Users\aashi\AppData\Local\Temp\ipykernel_17932\6722589.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_birds["Bird_Size"] = data_birds["WildlifeSpecies"].apply(map_size)
C:\Users\aashi\AppData\Local\Temp\ipykernel_17932\6722589.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_birds["Bird_Count"] = data_birds["NumberStruckActual"]
C:\Users\aashi\AppData\Local\Temp\ipykernel_17932\6722589.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T